In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import re
import requests
import pandas as pd

In [50]:
def job_offers_wttj(
        job_title: str = "data analyst",
        pages: int = 1
):
    # Instanciation de la liste contenant les liens pour les requêtes APIs.
    api_links = []
    # Lien de l'API de Welcome To The Jungle pour récupérer les données.
    api_link = f"https://api.welcometothejungle.com/api/v1/organizations"
    job = job_title.lower().replace(" ", "+")
    # Instanciation du driver Firefox.
    firefox_options = Options()
    firefox_options.headless = True
    driver = webdriver.Firefox(options=firefox_options)
    # Nom des colonnes à garder dans le dataframe final.
    url = f"https://www.welcometothejungle.com/fr/jobs?refinementList%5Boffices.country_code%5D%5B%5D=FR&query={job}&page=1"
    # Ouvre chaque page sur le navigateur.
    driver.get(url)
    try:
        page_numbers = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".sc-ezreuY.gGgoDq"))
        )
        page_max = page_numbers[-1].text
    except:
        print("Page number not found.")
    try:
        for i in range(1, pages+1):
            url = f"https://www.welcometothejungle.com/fr/jobs?refinementList%5Boffices.country_code%5D%5B%5D=FR&query={job}&page=1"
            # Ouvre chaque page sur le navigateur.
            driver.get(url)
            try:
                # Récupère le lien de chaque offre d'emploi sur la page.
                contents = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".sc-6i2fyx-0.gIvJqh"))
                )
                for content in contents:
                    link = content.get_attribute("href")
                    end_link = re.findall(r"/companies(.+)", link)[0]
                    full_link = api_link + end_link
                    # Rajoute le lien de chaque offre à la liste.
                    api_links.append(full_link)
            except Exception as e:
                print(f"Error scraping page {i} : {e}")
    finally:
        driver.quit()
        # Pour chaque lien de la liste, fait une requête API et stocke les informations dans un dataframe.
    return api_links, page_max

In [51]:
api_links, page_max = job_offers_wttj("data analyst", 1)
page_max

'25'

In [8]:
r = requests.get(api_links[0])
r.json()["job"]

{'recruitment_process': None,
 'status': 'published',
 'profession': {'category': {'cs': 'Tech',
   'en': 'Tech',
   'es': 'Tecnología',
   'fr': 'Tech',
   'po': 'Tech',
   'sk': 'Technológie'},
  'name': {'cs': 'Data Engineering',
   'en': 'Data Engineering',
   'es': 'Ingeniería de datos',
   'fr': 'Data Engineering',
   'po': 'Data Engineering',
   'sk': 'Data Engineering'}},
 'is_default': False,
 'contract_duration_min': None,
 'name': 'Margo Analytics - Data Engineer - H/F',
 'contract_type': 'FULL_TIME',
 'videos': [{'external_reference': '/M8Hd_oolT3I',
   'image': {'large': {'url': 'https://cdn-images.welcometothejungle.com/7QhWyhyMc0_IG67p9uZ1eDQmY3ElOLqUF4tNU9Svnek/rs:auto:1500::/q:85/czM6Ly93dHRqLXByb2R1Y3Rpb24vdXBsb2Fkcy92aWRlby9pbWFnZS80NTA0LzE2OTEwNS9tYXJnb192aWRlb193a3JQZDAxLnBuZw'},
    'medium': {'url': 'https://cdn-images.welcometothejungle.com/RkG58C8qk7MUPp_6WhEKHXa4cDypiOZhI550L9rt304/rs:auto:980::/q:85/czM6Ly93dHRqLXByb2R1Y3Rpb24vdXBsb2Fkcy92aWRlby9pbWFnZS80NTA0

In [6]:
from bs4 import BeautifulSoup
import pandas as pd
def clean_html(text):
    soup = BeautifulSoup(text, "html.parser")
    cleaned_text = soup.get_text(separator=" ")
    cleaned_text = cleaned_text.replace("\xa0", " ")
    return cleaned_text

df["description"] = df["description"].apply(clean_html)
df["organization.description"] = df["organization.description"].apply(clean_html)

In [7]:
df.to_csv("WTT_offers.csv")

In [4]:
df = pd.read_csv("WTT_offers.csv")

In [2]:
pd.set_option("display.max_columns", None)

In [7]:
df["application_fields.mode"]

KeyError: 'application_fields.mode'